In [1]:
import pandas as pd
import numpy as np
import sqlite3
import joblib
import os
import json
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
# === Додано Precision та Recall ===
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# --- Налаштування шляхів для збереження ---
MODELS_DIR = 'C:\\Data\\models'
LOGS_DIR = 'C:\\Data\\logs'
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

# --- 1. Зчитування даних з SQLite ---
database_file = 'C:\\Databases\\charge_database_alt.sqlite'
TABLE_SESSIONS = 'ChargingSessions_ScaledNoAnomaly'
TABLE_USERS = 'Users_Encoded'
TABLE_VEHICLES = 'Vehicles_ScaledNoAnomaly'

try:
    conn = sqlite3.connect(database_file)
    df_sessions = pd.read_sql_query(f"SELECT * FROM {TABLE_SESSIONS}", conn)
    df_users = pd.read_sql_query(f"SELECT * FROM {TABLE_USERS}", conn)
    df_vehicles = pd.read_sql_query(f"SELECT * FROM {TABLE_VEHICLES}", conn)
    conn.close()
    print(f"Дані успішно завантажено з {database_file}")
except Exception as e:
    print(f"Помилка завантаження даних з SQLite: {e}")
    exit() 

# --- 2. Об'єднання таблиць (Merging) ---
print("Об'єднання таблиць...")
df = pd.merge(df_sessions, df_users, on='user_id', how='left')
df = pd.merge(df, df_vehicles.drop(columns=['user_id'], errors='ignore'), on='vehicle_id', how='left')
print(f"Дані об'єднано. Загальна кількість рядків: {len(df)}")

# --- 3. Критична підготовка: Сортування за Часом ---
try:
    df['charging_start_time'] = pd.to_datetime(df['charging_start_time'])
    df = df.sort_values(by='charging_start_time').reset_index(drop=True)
    print("Дані відсортовано за 'charging_start_time'.")
except KeyError:
    print("ПОМИЛКА: Колонка 'charging_start_time' не знайдена.")
    exit()

# --- 4. Підготовка даних (X та Y) ---
def prepare_data(df):
    print("Підготовка фінальних X (ознак) та Y (цілі)...")
    try:
        conditions = [
            df['charger_type_Level 1'] == 1,
            df['charger_type_Level 2'] == 1,
            df['charger_type_DC Fast Charger'] == 1
        ]
        choices = ['Level 1', 'Level 2', 'DC Fast Charger']
        df['Target_ChargerType'] = np.select(conditions, choices, default='Other')
        df = df[df['Target_ChargerType'] != 'Other'] # Видаляємо неповні дані
        
        print(f"Розподіл цільової змінної (Y):\n{df['Target_ChargerType'].value_counts(normalize=True)}\n")

    except KeyError as e:
        print(f"ПОМИЛКА: Відсутня OHE колонка для створення цілі: {e}")
        exit()
    
    y_column = 'Target_ChargerType'
    columns_to_drop = [
        'session_id', 'user_id', 'vehicle_id', 'charging_station_id', 
        'charging_start_time', 'charging_end_time', 'timestamp', 
        'charging_station_location', 'time_of_day', 'day_of_week', 'vehicle_model',
        'energy_consumed_kwh', 'charging_duration_hours', 'charging_rate_kw', 
        'charging_cost_usd', 'state_of_charge_end',
        'charger_type_DC Fast Charger', 'charger_type_Level 1', 'charger_type_Level 2',
        'Target_ChargerType'
    ]
    
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    x_columns = df.columns.drop(existing_columns_to_drop)
    
    print(f"Цільова змінна (Y): {y_column}")
    print(f"Кількість вхідних ознак (X): {len(x_columns)}")
    
    X = df[x_columns]
    y = df[y_column]
    
    if X.isnull().sum().sum() > 0:
        print("\nПОПЕРЕДЖЕННЯ: Знайдено пропуски (NaN) в X. Заповнюю нулями...")
        X = X.fillna(0)
    
    return X, y

# Підготовка даних
X, y = prepare_data(df)

# --- 5. Ініціалізація Моделей ---
models = {
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1),
    "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1),
    "LightGBM": lgb.LGBMClassifier(random_state=42, n_jobs=-1),
    "SVM": SVC(random_state=42)
}

# --- 6. Ініціалізація TimeSeriesSplit ---
N_SPLITS = 5
tss = TimeSeriesSplit(n_splits=N_SPLITS)

# --- 7. Тренування моделей із TimeSeriesSplit (для оцінки) ---
model_training_logs = []
all_metrics_summary = {}

print("\n" + "="*30)
print(f"ПОЧАТОК КРОС-ВАЛІДАЦІЇ (TimeSeriesSplit, {N_SPLITS} зрізів)")
print("="*30)

for name, model in models.items():
    print(f"\nОцінка моделі: {name}")
    if name == "SVM":
        print("!!! (Це може зайняти значно більше часу) !!!")

    # === ВИПРАВЛЕННЯ: Ініціалізуємо fold_logs ТУТ ===
    fold_logs = []
    # ============================================

    # Додано precision та recall
    fold_metrics = {
        'accuracy': [], 
        'f1_weighted': [], 
        'precision_weighted': [], 
        'recall_weighted': []
    }

    for fold, (train_index, val_index) in enumerate(tss.split(X), 1):
        print(f"  Fold {fold}/{N_SPLITS}...")
        
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        # Розрахунок 4 метрик
        accuracy = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average='weighted', zero_division=0)
        precision = precision_score(y_val, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_val, y_pred, average='weighted', zero_division=0)
        
        fold_metrics['accuracy'].append(accuracy)
        fold_metrics['f1_weighted'].append(f1)
        fold_metrics['precision_weighted'].append(precision)
        fold_metrics['recall_weighted'].append(recall)
        
        fold_logs.append({
            'fold': fold,
            'status': 'completed',
            'training_timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'accuracy': accuracy,
            'f1_weighted': f1,
            'precision_weighted': precision,
            'recall_weighted': recall
        })
        # Вивід 4 метрик
        print(f"  Fold {fold} завершено. Acc: {accuracy:.4f}, Prec: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Усереднення 4 метрик
    avg_accuracy = np.mean(fold_metrics['accuracy'])
    avg_f1 = np.mean(fold_metrics['f1_weighted'])
    avg_precision = np.mean(fold_metrics['precision_weighted'])
    avg_recall = np.mean(fold_metrics['recall_weighted'])

    all_metrics_summary[name] = {
        'avg_accuracy': avg_accuracy,
        'avg_precision_weighted': avg_precision,
        'avg_recall_weighted': avg_recall,
        'avg_f1_weighted': avg_f1
    }

    model_training_logs.append({
        'model': name,
        'status': 'cross-validation completed',
        'avg_accuracy': avg_accuracy,
        'avg_precision_weighted': avg_precision,
        'avg_recall_weighted': avg_recall,
        'avg_f1_weighted': avg_f1,
        'fold_logs': fold_logs
    })
    print(f"Оцінка {name} завершена.")
    # Вивід 4 метрик
    print(f"Середні Метрики: Acc: {avg_accuracy:.4f}, Prec: {avg_precision:.4f}, Recall: {avg_recall:.4f}, F1: {avg_f1:.4f}")

# === ДОДАНО БЛОК ВИВОДУ МЕТРИК ===
print("\n" + "="*50)
print("ПІДСУМКОВІ СЕРЕДНІ МЕТРИКИ (з крос-валідації)")
print("="*50)
# Використовуємо json для красивого форматування
print(json.dumps(all_metrics_summary, indent=4))
print("="*50)
# ==================================

# --- 8. Збереження логів у JSON ---
training_log_path = os.path.join(LOGS_DIR, 'training_logs.json')
with open(training_log_path, 'w', encoding='utf-8') as f:
    json.dump(model_training_logs, f, indent=4, ensure_ascii=False)

print(f"\nЛоги крос-валідації збережено в {training_log_path}")

# --- 9. Фінальне Тренування та Збереження Моделей ---
print("\n" + "="*30)
print("ПОЧАТОК ФІНАЛЬНОГО ТРЕНУВАННЯ (на всіх даних)")
print("="*30)

for name, model in models.items():
    print(f"\nТренування фінальної моделі: {name}...")
    
    model.fit(X, y)
    
    model_filename = f"{name.replace(' ', '_').lower()}.joblib"
    model_path = os.path.join(MODELS_DIR, model_filename)
    joblib.dump(model, model_path)
    print(f"Модель {name} натренована і збережена в {model_path}")

print("\n" + "="*30)
print("РОБОТУ ЗАВЕРШЕНО")
print("="*30)
print(f"Натреновані моделі збережено в директорії {MODELS_DIR}")

Дані успішно завантажено з C:\Databases\charge_database_alt.sqlite
Об'єднання таблиць...
Дані об'єднано. Загальна кількість рядків: 1320
Дані відсортовано за 'charging_start_time'.
Підготовка фінальних X (ознак) та Y (цілі)...
Розподіл цільової змінної (Y):
Target_ChargerType
Level 1            0.347727
Level 2            0.326515
DC Fast Charger    0.325758
Name: proportion, dtype: float64

Цільова змінна (Y): Target_ChargerType
Кількість вхідних ознак (X): 8

ПОЧАТОК КРОС-ВАЛІДАЦІЇ (TimeSeriesSplit, 5 зрізів)

Оцінка моделі: Logistic Regression
  Fold 1/5...
  Fold 1 завершено. Acc: 0.3773, Prec: 0.3781, Recall: 0.3773, F1: 0.3635
  Fold 2/5...
  Fold 2 завершено. Acc: 0.3818, Prec: 0.3791, Recall: 0.3818, F1: 0.3789
  Fold 3/5...
  Fold 3 завершено. Acc: 0.3136, Prec: 0.3041, Recall: 0.3136, F1: 0.3016
  Fold 4/5...
  Fold 4 завершено. Acc: 0.3182, Prec: 0.3206, Recall: 0.3182, F1: 0.3067
  Fold 5/5...
  Fold 5 завершено. Acc: 0.2864, Prec: 0.2927, Recall: 0.2864, F1: 0.2837
Оцінка 